## Encapsulation

Another very useful practice in OOP is _Encapsulation_. Encapsulation is the process of wrapping (or __encapsulating__) similar concerns and data into a large construct. Often, you will encounter both abstraction and encapsulation consecutively. The difference is that abstraction shows the main functionality of certain pieces of code without focusing on the internal structure, whereas encapsulation involves grouping related functionalities into a relatively large construct. 

> __Encapsulation__ is the process of grouping related functionalities into a large construct.


### Abstraction vs encapsulation
Abstraction and encapsulation are often confused, particular on the first encounter. To help you understand the differences between them, we will see an example. Previously, we explored the AnimalScraper class, which was abstracted from the rest of the code. At first glance, this appears similar to encapsulation, since we are grouping a bunch of methods. However, the keyword here is _related_. 

Certainly, AnimalScraper grouped some methods; however, these methods are only related in that they form a pipeline together (a series of steps). Nevertheless, _related_ methods do not necessarily have to be working in tandem. They can work separately, and in such a case, we orchestrate them using encapsulation.

Let us define two new functions: get_taxonomy and get_class. 

1. get_taxonomy will obtain a list of zoological synonyms (thus, you might find that animal on another webpage using a different name).
2. get_class will obtain the animal class (mammals, birds, amphibians, reptiles or fish).

In [ ]:
import re
import requests
from typing import List
from bs4 import BeautifulSoup

def get_class(animal:str) -> str:
    ROOT = 'https://en.wikipedia.org/wiki/'
    r = requests.get(ROOT + animal)
    soup = BeautifulSoup(r.content, 'html.parser')
    class_row = soup.find('td', text = re.compile('Class:'))
    animal_class = class_row.find_next_sibling().text.strip()
    return animal_class

def get_taxonomy(animal:str) -> List:
    ROOT = 'https://en.wikipedia.org/wiki/'
    r = requests.get(ROOT + animal)
    print(type(r.text))
    soup = BeautifulSoup(r.content, 'html.parser')
    print(type(soup))
    syn_text = soup.find('a', text = re.compile('Synonyms'))
    if syn_text:
        syn_header = syn_text.find_parent('tr')
        syn_table = syn_header.find_next_sibling()
        contents = syn_table.find_all('i')
        if contents:
            contents = [x.text for x in contents]
            return contents
    else:
        return []

print(get_class('koala'))
print(get_taxonomy('koala'))


Notice that these functions are independent of one another; however, their concerns are in the same field (extracting information about the animal). Thus, we could group them under the same class so that the next time we require information about an animal, we can use the corresponding method from that class.

In [ ]:
class AnimalReporter:
   
    def __init__(self, animal: str):
        self.animal = animal
    
    def _say_hello_protected(self):
        print('Hi, Im a protected method')
    
    def say_hello_public(self):
        print('Hi, Im a public method')
        self._say_hello_priv()

    def get_class(self) -> str:
        ROOT = 'https://en.wikipedia.org/wiki/'
        r = requests.get(ROOT + self.animal)
        soup = BeautifulSoup(r.content, 'html.parser')
        class_row = soup.find('td', text = re.compile('Class:'))
        animal_class = class_row.find_next_sibling().text.strip()
        return animal_class
    
    def get_taxonomy(self):
        ROOT = 'https://en.wikipedia.org/wiki/'
        r = requests.get(ROOT + self.animal)
        soup = BeautifulSoup(r.content, 'html.parser')
        syn_text = soup.find('a', text = re.compile('Synonyms'))
        if syn_text:
            syn_header = syn_text.find_parent('tr')
            syn_table = syn_header.find_next_sibling()
            contents = syn_table.find_all('i')
            if contents:
                contents = [x.text for x in contents]
                return contents
        else:
            return []

ar = AnimalReporter('koala')

In [ ]:
ar._say_hello_protected()

You may have noticed the presence of an underscore. One of the benefits of encapsulation is privacy. Protected variables and methods can be defined so that the user cannot access them. In Python, this is technically not true: you cannot have a protected method. However, there is a convention: if a method has a prefixed underscore, it should not be changed (they trust that you will not change it). These protected methods are (or should be) only accessible within the class or the module, as we will see later.

To achieve an increased level of privacy, private methods can be defined by adding two underscores. This will ensure that the attribute or method is inaccessible to the user and only accessible within the class.

>Encapsulation sets boundaries for your methods so that they are private and only accessible within the class or module.

Conversely, public methods are also called __interfaces__ because they are accessible to the public. 

Think about encapsulation like building walls around your class. Private/protected methods will be within the walls, while public methods will be the gates for getting access to those private/protected methods.

Experiment on this to improve your understanding. 

_Tip_: read the provided type hint to know what type of variables to return.

In [ ]:
from bs4 import BeautifulSoup
from typing import Union
from typing import List
import requests
import re

class AnimalReporter:
    def __init__(self, animal: str):
        self.animal = animal
    
    def _get_request(self) -> Union[bytes, str]:
        ROOT = 'https://en.wikipedia.org/wiki/'
        r = requests.get(ROOT + self.animal)
        return r.text

    def _get_soup(self, html: Union[bytes, str]) -> BeautifulSoup:
        soup = BeautifulSoup(html, 'html.parser')
        return soup
        
    def get_class(self) -> str:
        html = self._get_request()
        soup = self._get_soup(html)
        class_row = soup.find('td', text = re.compile('Class:'))
        animal_class = class_row.find_next_sibling().text.strip()
        return animal_class
    
    def get_taxonomy(self) -> List:
        html = self._get_request()
        soup = self._get_soup(html)
        syn_text = soup.find('a', text = re.compile('Synonyms'))
        if syn_text:
            syn_header = syn_text.find_parent('tr')
            syn_table = syn_header.find_next_sibling()
            contents = syn_table.find_all('i')
            if contents:
                contents = [x.text for x in contents]
                return contents
        else:
            return []

ar = AnimalReporter('koala')
ar.get_class()

## Abstraction and Encapsulation 

Now we have two classes: AnimalScraper and AnimalReporter, which are related in that we can refer to one of them if we need data about an animal. However, grouping them into the same class would be quite inefficient and vague. Instead, we can use a module to gather them into a script. Modules are even higher-level than classes, and they apply a type of encapsulation since they group multiple related classes and functions.

> Modules apply a type of encapsulation that groups related functions or classes.

<p align=center><img src=images/animal_module.png width=400></p>

Notice that we employed both abstraction and encapsulation to create this module. Conventionally, abstraction and encapsulation work together by grouping related functionalities and concealing the parts that do not matter to the user. This will allow us to change the internal code rapidly without affecting the output.

If the difference is still not clear, here is a summary:

<p align=center><img src=images/Abstraction_vs_Encapsulation.png?modified=232 width=400></p>

# Summary

- Abstraction is a tool to hide complexity, the user is not aware of the implementation details.
- Encapsulation is a tool to group related functionalities together.


# Practical

## Mixin class for private methods
1. Create a mixin class named AsDictMixin 
2. This class will be just inherited, so don't use a constructor for it
3. You just need to define the following method: `to_dict(self)` which returns a `dict` representation of the object that inherits this mixin class.
4. You might want to use the `__dict__` method, which returns a dictionary representation of an object.
5. The class should look like this:

In [ ]:
class AsDictMixin:
    def to_dict(self):
        ### Your code here
        pass
    def _represent(self, value):
        if isinstance(value, object):
            if hasattr(value, 'to_dict'):
                return value.to_dict()
            else:
                return str(value)
        else:
            return value

    def _is_internal(self, prop):
        return prop.startswith('_')

#### So when running the following code, the to_dict() method doesn't return private attributes.

```
class Person(AsDictionaryMixin):
    def __init__(self, name, address, salary):
        self.name = name
        self.address = address
        self._salary = salary

ivan = Person('Ivan', 'London', '100000000')
ivan.to_dict()
```

{'name': 'Ivan', 'address': 'London'} (No salary is shown, because it's private)

# Assessment

### 1. Look information about modules, packages, and how they are organized (we will see more on this in next sections, so just read about them)
### 2. How does encapsulation benefit from modules? 
### 3. How does encapsulation benefit from packages?